In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monthly-food-price-estimates/WLD_RTP_details_2023-10-02.csv
/kaggle/input/monthly-food-price-estimates/ddi-documentation-english_microdata-4483 (1).pdf
/kaggle/input/monthly-food-price-estimates/ddi-documentation-english_microdata-4509.pdf
/kaggle/input/monthly-food-price-estimates/WLD_RTFP_country_2023-10-02.csv
/kaggle/input/spark4ai-prediction/soil_dataset/sample_submission.csv
/kaggle/input/spark4ai-prediction/soil_dataset/train.csv
/kaggle/input/spark4ai-prediction/soil_dataset/test.csv


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df=pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/train.csv')
df_part1 = df.iloc[:, :23]
df_part2=df.iloc[:,23:]
df_part1.drop('Id',axis=1)
fd=df.head(101)
fx=pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/test.csv')
fx_part1 = df.iloc[:, :23]
fx_part2=df.iloc[:,23:]
fx_part1.drop('Id',axis=1)


,Property_A,Property_B,Property_C,Property_D,Property_E,Property_F,A,B,C,D,...,G,H,I,J,K,L,M,N,O,P
0,5.99,0.115797,1.638,49.0,19.0,32.0,101745.9975,292211.0199,38816.97313,5308.374022,...,384.928482,52565.24979,50.560870,50.918543,96.767202,9.663707,200.672050,71.013243,34,28.406635
1,5.15,0.183483,1.482,47.0,16.0,37.0,105712.6402,315908.0751,39888.72776,5431.548180,...,372.064549,50665.24752,53.176087,49.900172,93.678887,9.663707,199.593168,73.133041,35,29.386174
2,5.55,0.173130,1.404,50.0,20.0,30.0,102621.3309,304334.3675,42120.98104,5513.664286,...,422.530751,52748.36917,52.304348,50.918543,100.884955,9.663707,211.460870,74.192941,37,31.345253
3,5.09,0.196011,1.638,50.0,15.0,35.0,113446.2881,311356.2657,39195.84181,5428.306755,...,396.802883,53893.69764,52.304348,49.900172,97.796640,10.542226,209.303106,69.953344,35,30.365713
4,4.73,0.203319,1.677,53.0,19.0,28.0,109714.6033,322110.4501,38347.65382,5431.548180,...,346.336681,48784.11209,49.689130,44.808317,99.855517,8.785189,199.593168,73.133041,34,27.427096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,5.10,1.550000,0.980,0.5,57.1,42.2,116328.7458,312309.0848,43708.13360,5196.004615,...,318.629746,55262.09881,65.380435,49.900172,115.297092,10.542226,231.959627,138.846789,39,35.263409
246,4.90,3.090000,1.060,5.1,59.6,35.3,101064.1588,285120.0053,45718.52685,5233.821242,...,421.541217,55073.43036,54.919565,52.955284,108.091023,11.420745,230.880745,91.151327,33,33.304331
247,4.90,0.620000,0.860,0.1,57.5,42.4,100451.4254,308552.1142,44022.15089,5112.808034,...,314.671613,47369.09871,46.202174,44.808317,97.796640,7.906670,213.618633,80.552336,32,31.345253
248,5.10,2.960000,0.850,1.9,68.8,29.3,111843.0457,329063.2319,45397.68311,5238.143143,...,342.378547,49069.33439,47.073913,47.863430,95.737764,10.542226,219.013044,117.648806,33,30.365713


In [3]:
from sklearn.metrics import mean_squared_error
model=RandomForestRegressor()
Y=df_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']]

X=df_part1.drop(['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F','Id'],axis=1)
X_test=fx_part1.drop(['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F','Id'],axis=1)
Y_test=fx_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']]


model.fit(X, Y)
y_pred= model.predict(X_test)
y_pred1=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=y_pred[i][j]
    y_pred1.append(a)    
predict1=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")
          




Root Mean Squared Error (RMSE): 2.270074647954372


In [4]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
model = xgb.XGBRegressor()
model.fit(X, Y)
y_pred = model.predict(X_test)

mae = mean_absolute_error(Y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
y_pred1=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=y_pred[i][j]
    y_pred1.append(a)    
predict2=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")
        


Mean Absolute Error: 0.0009236658513924428
Root Mean Squared Error (RMSE): 0.0014996411013326915


In [5]:
X1=df_part2
model=RandomForestRegressor()
Y1=df_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']] 
X1_test=fx_part2
Y1_test=fx_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']]

model.fit(X1, Y1)
predictions = model.predict(X1_test)
y_pred2=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=predictions[i][j]
    y_pred2.append(a)    
predict3=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y1_test,predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")




Root Mean Squared Error (RMSE): 2.2425068170172824


In [6]:
model = xgb.XGBRegressor()
model.fit(X1, Y1)
predictions = model.predict(X1_test)
y_pred2=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=predictions[i][j]
    y_pred2.append(a)    
predict4=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y1_test,predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")



Root Mean Squared Error (RMSE): 0.0021927640237982306


In [7]:
bias_1 = 1.97
bias_2 = 0.03
fused_data = (bias_1 * predict2 + bias_2 * predict4) / (bias_1 + bias_2)
fused_data['Id']=predict2['Id']
fused_data





,Id,Predicted
0,1,107.743089
1,2,106.821602
2,3,107.124584
3,4,106.922068
4,5,106.609921
...,...,...
96,97,109.208857
97,98,108.683509
98,99,109.170455
99,100,108.444152


In [8]:
csv_file_path = ' my_dataframe.csv' 
fused_data.to_csv(csv_file_path, index=False)